<a href="https://colab.research.google.com/github/Thanadett/Agriculture_Robot_Auto/blob/main/Yolo11n_PotEgde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ติดตั้ง Librarry ที่จำเป็น

In [ ]:
!pip install roboflow
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 138.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00


# นำเข้า DataSet จาก Roboflow


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wOScTYAj4LXlpRxgF2TJ")
project = rf.workspace("class-zkual").project("test-ks2dh")
version = project.version(4)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Test-4 in yolov11:: 100%|██████████| 1014/1014 [00:00<00:00, 6993.05it/s]


# Train Model


In [ ]:
%%bash
mkdir -p /content/Test-4/val/images /content/Test-4/val/labels
mkdir -p /content/Test-4/test/images /content/Test-4/test/labels

ls /content/Test-4/train/images | head -n 100 > val_files.txt

while read f; do
  mv /content/Test-4/train/images/$f /content/Test-4/val/images/
  mv /content/Test-4/train/labels/${f%.*}.txt /content/Test-4/val/labels/
done < val_files.txt

ls /content/Test-4/train/images | head -n 50 > test_files.txt

while read f; do
  mv /content/Test-4/train/images/$f /content/Test-4/test/images/
  mv /content/Test-4/train/labels/${f%.*}.txt /content/Test-4/test/labels/
done < test_files.txt

rm val_files.txt test_files.txt

In [ ]:
!ls /content/Test-4/train/images | wc -l
!ls /content/Test-4/train/labels | wc -l

!ls /content/Test-4/val/images | wc -l
!ls /content/Test-4/val/labels | wc -l

!ls /content/Test-4/test/images | wc -l
!ls /content/Test-4/test/labels | wc -l

354
354
100
100
50
50


In [ ]:
!yolo detect train \
  model=yolo11n.pt \
  data=Test-4/data.yaml \
  imgsz=640 \
  epochs=300 \
  patience=100 \
  batch=16

New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.251 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Test-4/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimiz

In [ ]:
!yolo val \
  model=/content/runs/detect/train6/weights/best.pt \
  data=/content/Test-4/data.yaml \
  split=val


Ultralytics 8.3.251 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/__init__.py", line 987, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 612, in val
    validator(model=self.model)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/validator.py", line 175, in __call__
    self.data = check_det_dataset(self.args.data)
                ^^^^^^^^^^^^^^^^^^^^